# Threads para processar em tempo real

A ideia aqui é rodar o sistema em tempo real antes de passar pra um .py msm.

In [1]:
import sys
import pyaudio
import numpy as np
import warnings
import threading
from time import time, sleep

sys.path.append("/home/dimi/Programming/IC2019/Raspberry/Python/")

from delayAndSum import delayAndSum as DaS
from extracaoFeatures import extrairFeaturesUnicoFrame as ExtrairFeatures
from iniciarNormalizadorEClassificador import main as IniciarObjetos

warnings.filterwarnings("ignore")

### Verificando o dispositivo que vamos usar pra gravar

In [2]:
def listarDispositivos():
    p = pyaudio.PyAudio()
    info = p.get_host_api_info_by_index(0)
    numdevices = info.get('deviceCount')

    for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

In [3]:
listarDispositivos()

Input Device id  3  -  HDA Intel PCH: ALC282 Analog (hw:1,0)
Input Device id  5  -  pulse
Input Device id  6  -  default


### Definição de funções

##### Função para realizar todo o processamento em uma janela

A função receberá uma janela gravada, aplicará o **Delay and Sum**, extrairá as **features** do sinal beamformado, **normalizará** o vetor de features obtido, e irá **classificar** o dado gerado.

In [4]:
def processarJanela(arraySinais, idJanela, freqAmostragem, objNormalizador, objClassificador, verbose=True):
    
    # VERBOSE
    if verbose == True:
        # INICIANDO A MEDICAO DE TEMPO
        tempoInicio = time()
        stringID = "(id:" + str(idJanela) + ")"
        print(stringID, "Iniciando o processamento")      
    
    # DELAY AND SUM
    sinalDaS = DaS(arraySinais)

    # EXTRACAO DE FEATURES
    features, __ = ExtrairFeatures(sinalDaS, freqAmostragem)

    # NORMALIZANDO AS FEATURES
    features = objNormalizador.transform([features])[0]

    # CLASSIFICACAO
    predicao = objClassificador.predict([features])[0]
    
    # VERBOSE
    if verbose == True:
        # FINALIZANDO A MEDICAO DE TEMPO
        tempoFim = time()        
        print(stringID, "Classificado como", predicao)
        print(stringID, "Processamento finalizado em", tempoFim - tempoInicio, "s")

##### Função para gravar uma janela com o dispositivo

A função abaixo deverá gravar uma janela e depois enviá-la para a função anterior. A função anterior deve ser rodada como uma thread.

In [9]:
def gravarJanela(idJanela, objNormalizador, objClassificador, idDispositivoGravacao, tempoJanela=0.2, freqAmostragem=16000, profundidadeBytes=2, qtdCanais=4, verbose=True):

    assert profundidadeBytes == 2, "Profundidade de bytes diferente de 2! Você deve alterar o sistema manualmente."
    
    # DEFININDO O TAMANHO DA JANELA EM AMOSTRAS
    tamanhoJanela = int(tempoJanela * freqAmostragem)
    
    # INSTANCIANDO UM OBJ PY AUDIO E MANDANDO OS PARAMETROS INICIAIS
    objPyAudio = pyaudio.PyAudio()
    stream = objPyAudio.open(
        input_device_index = idDispositivoGravacao,
        rate               = freqAmostragem,
        format             = objPyAudio.get_format_from_width(profundidadeBytes),
        channels           = qtdCanais,
        input              = True    
    )
    
    # VERBOSE 
    if verbose == True:
        stringID = "(id:" + str(idJanela) + ")"
        print(stringID, "Iniciando gravação. Timestamp:", time())

    # GRAVANDO A JANELA
    janelaBinaria = stream.read(tamanhoJanela)
    
    # VERBOSE 
    if verbose == True:
        print(stringID, "Gravação finalizada. Timestamp:", time())

    # MATANDO OS OBJETOS PRA LIMPAR MEMORIA
    stream.stop_stream()
    stream.close()
    objPyAudio.terminate()
        
    # CONVERTO A JANELA PRA INT 16        
    janelaInt16 = np.fromstring(janelaBinaria, dtype=np.int16)
    
    # COMO TEM OS 4 MICS NA JANELA, VOU DAR UM RESHAPE PRA CADA MIC FICAR EM UMA LINHA
    janelaInt16 = janelaInt16.reshape((tamanhoJanela, qtdCanais)).T
    
    # GARANINDO QUE A DIMENSIONALIDADE ESTA CORRETA
    assert janelaInt16.shape[0] == qtdCanais and janelaInt16.shape[1] == tamanhoJanela, "Erro na dimensionalidade da janela gravada!"
    
    # MANDANDO A JANELA GRAVADA APRA O PROCESSAMENTO
    objThread = threading.Thread(target=processarJanela, args=(janelaInt16, idJanela, freqAmostragem, objNormalizador, objClassificador, verbose))
    objThread.start()

### Parâmetros iniciais

In [6]:
idDispositivoGravacao = 6
tempoJanela           = 0.250
freqAmostragem        = 16000
profundidadeBytes     = 2
qtdCanais             = 4
caminhoCSVDataset     = "/home/dimi/Downloads/datasets/SESA_v2_"+str(int(freqAmostragem/1000))+"kHz_"+str(int(profundidadeBytes*8))+"bits/SESA_v2_"+str(int(freqAmostragem/1000))+"kHz_"+str(int(profundidadeBytes*8))+"bits_"+str(int(tempoJanela*1000))+"ms_58features_desescalonado_remocaoSilencio.csv"

### Rodando

In [7]:
objClassificador, objNormalizador = IniciarObjetos(caminhoCSVDataset, classificador=None, verbose=True)

Carregando o CSV do dataset
CSV carregado
Separando o que é data e target
Iniciando objeto normalizador
Treinando objeto normalizador
Normalizando os dados de treinamento
Instanciando objeto classificador
Treinando o classificador
objClassificador e objNormalizador prontos: operação finalizada.


In [21]:
i = 1
tempoJanelaTempoReal = tempoJanela

while i < 20:
    
    # INICIANDO JANELA PRINCIPAL
    objThread = threading.Thread(target=gravarJanela, args=(i, objNormalizador, objClassificador, idDispositivoGravacao, tempoJanelaTempoReal, freqAmostragem, profundidadeBytes, qtdCanais))
    objThread.start()
    
    # DELAY PARA COMECAR A JANELA DE SOBREPOSICAO
    i += 1
    sleep(tempoJanelaTempoReal/2)
    
    # INICIANDO JANELA DE SOBREPOSICAO
    objThread = threading.Thread(target=gravarJanela, args=(i, objNormalizador, objClassificador, idDispositivoGravacao, tempoJanelaTempoReal, freqAmostragem, profundidadeBytes, qtdCanais))
    objThread.start()
    
    # DELAY PARA COMECAR A JANELA PRICIPAL NA PROXIMA ITERACAO
    i += 1
    sleep(tempoJanelaTempoReal/2)

(id:1) Iniciando gravação. Timestamp: 1594841862.8561976
(id:2) Iniciando gravação. Timestamp: 1594841862.982742
(id:3) Iniciando gravação. Timestamp: 1594841863.1074574
(id:1) Gravação finalizada. Timestamp: 1594841863.168746
(id:1) Iniciando o processamento
(id:1) Classificado como siren
(id:1) Processamento finalizado em 0.03289508819580078 s
(id:4) Iniciando gravação. Timestamp: 1594841863.2322772
(id:2) Gravação finalizada. Timestamp: 1594841863.2949812
(id:2) Iniciando o processamento
(id:2) Classificado como siren
(id:2) Processamento finalizado em 0.03989005088806152 s
(id:5)(id:3) Iniciando gravação. Timestamp: 1594841863.3598154
 Gravação finalizada. Timestamp: 1594841863.3600209
(id:3) Iniciando o processamento
(id:3) Classificado como siren
(id:3) Processamento finalizado em 0.04546236991882324 s
(id:4) Gravação finalizada. Timestamp: 1594841863.4803414
(id:6) Iniciando gravação. Timestamp: 1594841863.4839833
(id:4) Iniciando o processamento
(id:4) Classificado como siren
(